### V2 Recommendations

In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd
from calitp_data_analysis.sql import to_snakecase

In [ ]:
import altair as alt
import _threshold_utils as threshold_utils

from shared_utils import calitp_color_palette as cp
from shared_utils import geography_utils, rt_utils, styleguide, utils

In [ ]:
# import intake
# catalog = intake.open_catalog("./catalog_threshold.yml")

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [ ]:
route_length, time_segments, valid_stats = threshold_utils.load_dataframes(threshold_utils.TIME_CUTOFFS, threshold_utils.SEGMENT_CUTOFFS)

In [ ]:
threshold_utils.create_operator_visuals(500,400)

In [ ]:
threshold_utils.operator_brush(route_length)

### Statewide
#### Statewide Routes by Cutoff

In [ ]:
threshold_utils.create_statewide_visuals(500,400)

### Recs - 15 minutes, 70% of segments

In [ ]:
time_segments.trip_time.median()

In [ ]:
leniency_strigent = threshold_utils.valid_stats_leniency(valid_stats)

In [ ]:
f"Median % of trips kept if the most lenient threshold is applied {leniency_strigent.Max.median()}%"

In [ ]:
f"Median % of trips kept if the most stringent threshold is applied {leniency_strigent.Min.median()}%"

### Find specific missing routes after applying thresholds

In [ ]:
threshold_utils.find_cut_routes(15, 0.7)